In [21]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import numpy as np
from itertools import islice
from numpy import random

In [2]:
df = pd.read_csv('data/cleaned/dataframe')
df.set_index('Unnamed: 0', inplace=True)
df.index.names = ['date']
df.head()

,TSLA,MSFT,GOOG,PFE,AAPL,AMZN,SPY,ICLN,XOM,WBA
date,,,,,,,,,,
2010-06-29,4.778,18.280058,226.281845,9.088278,7.880552,108.610001,84.502396,11.210094,38.263123,20.278431
2010-06-30,4.766,18.044788,221.644226,9.075546,7.737815,109.260002,83.699646,11.171836,38.116188,20.439186
2010-07-01,4.392,18.162430,218.924423,9.056454,7.643986,110.959999,83.326614,11.393739,37.808968,20.332008
2010-07-02,3.840,18.248686,217.459900,8.999177,7.596610,109.139999,82.872505,11.592691,37.782242,20.178911
2010-07-06,3.222,18.680012,217.220810,9.094641,7.648601,110.059998,83.415825,11.745731,38.376659,20.377951


In [3]:
dates = sorted(df.index.values)
last_5pct = sorted(df.index.values)[-int(0.05*len(dates))]
df_val = df[(df.index >= last_5pct)]
df_train = df[(df.index < last_5pct)]

In [29]:
b = 30
f = 5
w = b + f
sequential_data = []
prev_days = deque(maxlen=w)
for col in df_train.columns:
    df = df_train[col].pct_change()
    df.dropna(inplace=True)
    seq = []
    for i in df.values:
        prev_days.append(i)
        if len(prev_days) == w:
            pd = np.array(list(prev_days))
            b_seq = pd[:b]
            b_min_max = max((abs(b_seq.min()), abs(b_seq.max())))
            b_seq = np.interp(b_seq, (-b_min_max, b_min_max), (-1, +1))
            f_seq = np.interp(pd[-f:], (-b_min_max, b_min_max), (-1, +1))
            seq.append([np.array(b_seq), np.array(f_seq)])
    sequential_data.append(seq)
for i in range(len(sequential_data)):
    random.shuffle(sequential_data[i])
sequential_data = np.array(sequential_data)

In [31]:
print(len(sequential_data)) # Assets
print(len(sequential_data[1])) # Sequences
print(len(sequential_data[2][2])) # b, f
print(len(sequential_data[3][3][0])) # b
print(len(sequential_data[3][3][1])) # f
print(sequential_data[3][3][0])

10
2274
2
30
5
[-0.23848881 -1.          0.38343267  0.14913333  0.10803994 -0.22891692
 -0.31160378 -0.10927355 -0.06849458  0.43917479  0.21706243  0.29677497
  0.12046417  0.440303    0.05275492 -0.02633031 -0.14498574  0.
 -0.25140036  0.79915418  0.33916965  0.09049669 -0.18056963 -0.1166345
  0.23399513  0.2196452   0.16698769 -0.25787183 -0.1557934   0.33893303]


In [32]:
np.save('data/preprocessed/prices', sequential_data)